In [157]:
import os
import numpy as np

base_path = '/mnt/disk1/heonseok/MPMLD/output'

In [158]:
dataset = 'location'

z_dim_list = [
    # '16',
    '64',
]

setsize_list = [
    # '500',
    # '1000',
    '2000',
]

lr_list = [
    '0.001',
    # '0.01',
    # '0.1',
]

ref_list = [
    '0.1',
]

arc_list = [
    # 'A',
    # 'B',
    'C',
]

dis_type_list = [
    # 'base',
    'type5',
]

cw_list = [
    '0.01',
    '0.1',
    '1.0',
]

mw_list = [
    '0.01',
    '0.1',
    '1.0',
]

recon_model_list = []
for z_dim in z_dim_list:
    for setsize in setsize_list:
        for lr in lr_list:
            for ref in ref_list:
                for arc in arc_list:
                    for dis_type in dis_type_list:
                        if 'base' in dis_type:
                            recon_model = 'AE_z{}_setsize{}_lr{}_ref{}_arc{}_{}'.format(
                                z_dim, setsize, lr, ref, arc, dis_type,
                            )
                            recon_model_list.append(recon_model)
                            continue
                        for cw in cw_list:
                            for mw in mw_list:
                                recon_model = 'AE_z{}_setsize{}_lr{}_ref{}_arc{}_{}_cw{}_mw{}'.format(
                                    z_dim, setsize, lr, ref, arc, dis_type, cw, mw,
                                )
                                recon_model_list.append(recon_model)

for recon_model in recon_model_list:
    print('\''+recon_model+'\',')

class_model_list = []
for idx in range(len(recon_model_list)):
    class_model_list.append(recon_model_list[idx] + '_FCNClassifierA')
    print('\''+class_model_list[idx]+'\',')

'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0_FCNClassifierA',

In [169]:
for recon_model in recon_model_list:
    print('\''+recon_model+'\',')

for recon_model in recon_model_list:
    recon_path = os.path.join(base_path, dataset, 'reconstructor', recon_model)

    # class_acc_full_list = []
    # class_acc_content_list = []
    # class_acc_style_list = []
    # membership_acc_full_list = []
    # membership_acc_content_list = []
    # membership_acc_style_list = []

    for repeat in range(5):
        try:
            recon_repeat_path = os.path.join(recon_path, 'repeat{}'.format(repeat))
            recon_mse_path = os.path.join(recon_repeat_path, 'mse.npy')
            recon_mse = np.load(recon_mse_path, allow_pickle=True)
            # print(recon_mse)

            if repeat == 0:
                mse_list = recon_mse
            else:
                mse_list += recon_mse
            # class_acc_full_list.append(recon_acc['class_acc_full'])
            # class_acc_content_list.append(recon_acc['class_acc_content'])
            # class_acc_style_list.append(recon_acc['class_acc_style'])
            # membership_acc_full_list.append(recon_acc['membership_acc_full'])
            # membership_acc_content_list.append(recon_acc['membership_acc_content'])
            # membership_acc_style_list.append(recon_acc['membership_acc_style'])

        except FileNotFoundError:
            # print('File not found: ', recon_acc_path)
            continue

    mse_list = mse_list/5

    result_list = []
    for idx, mse in enumerate(mse_list):
        result_list.append('{:.4f}'.format(mse))
    print(','.join(result_list))

    # result_list = [
    #     *recon_model.split('_'),
    #     np.average(class_acc_full_list), np.average(class_acc_content_list), np.average(class_acc_style_list),
    #     np.average(membership_acc_full_list), np.average(membership_acc_content_list), np.average(membership_acc_style_list)
    # ]
    #
    # for idx, result in enumerate(result_list):
    #     if not type(result) is str:
    #         result_list[idx] = '{:.4f}'.format(result)
    # print(','.join(result_list))

'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw1.0',
0.0370,0.0538,0.0531,0.1401,0.1421,0.1427,0.1483,0.1533,0.1525
0.0362,0.0531,0.0526,0.1551,0.1569,0.1575,0.1747,0.1780,0.1782
0.0373,0.0540,0.0536,0.1603,0.1640,0.1635,0.1240,0.1264,0.1259
0.0371,0.0531,0.0526,0.1704,0.1726,0.1732,0.1671,0.1710,0.1690
0.0376,0.0530,0.0524,0.1664,0.1682,0.1670,0.1996,0.2013,0.2022
0.0370,0.0534,0.0529,0.1983,0.2014,0.2016,0.1228,0.1251,0.1238
0.0347,0.0522,0.0517,0.1563,0.1582,0.1587,0.1600,0.1642,0.1632
0.0372,0.0534

In [164]:
for recon_model in recon_model_list:
    print('\''+recon_model+'\',')

for recon_model in recon_model_list:
    recon_path = os.path.join(base_path, dataset, 'reconstructor', recon_model)

    class_acc_full_list = []
    class_acc_content_list = []
    class_acc_style_list = []
    membership_acc_full_list = []
    membership_acc_content_list = []
    membership_acc_style_list = []

    for repeat in range(5):
        try:
            recon_repeat_path = os.path.join(recon_path, 'repeat{}'.format(repeat))
            recon_acc_path = os.path.join(recon_repeat_path, 'acc.npy')
            recon_acc = np.load(recon_acc_path, allow_pickle=True).item()

            class_acc_full_list.append(recon_acc['class_acc_full'])
            class_acc_content_list.append(recon_acc['class_acc_content'])
            class_acc_style_list.append(recon_acc['class_acc_style'])
            membership_acc_full_list.append(recon_acc['membership_acc_full'])
            membership_acc_content_list.append(recon_acc['membership_acc_content'])
            membership_acc_style_list.append(recon_acc['membership_acc_style'])

        except FileNotFoundError:
            # print('File not found: ', recon_acc_path)
            continue

    result_list = [
        *recon_model.split('_'),
        np.average(class_acc_full_list), np.average(class_acc_content_list), np.average(class_acc_style_list),
        np.average(membership_acc_full_list), np.average(membership_acc_content_list), np.average(membership_acc_style_list)
    ]

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw1.0',
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw0.01,0.8648,0.7384,0.1299,0.9468,0.8252,0.8891
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw0.1,0.8575,0.7211,0.1383,0.8347,0.5001,0.8200
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw1.0,0.8519,0.7232,0.1353,0.6627,0.5000,0.8039
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.1,mw0.01,0.8434,0.7628,0.0671,0.9149,0.8406,0.8485
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.1,mw

In [ ]:
for class_model in class_model_list:
    print('\''+class_model+'\',')

for class_model in class_model_list:
    class_path = os.path.join(base_path, dataset, 'classifier', class_model)

    result_list = []
    for recon_type in ['full_z', 'content_z', 'style_z']:
        train_acc_list = []
        valid_acc_list = []
        test_acc_list = []
        for repeat in range(5):
            try:
                class_repeat_path = os.path.join(class_path, recon_type, 'repeat{}'.format(repeat))
                class_acc = np.load(os.path.join(class_repeat_path, 'acc.npy'), allow_pickle=True).item()
                train_acc_list.append(class_acc['train'])
                valid_acc_list.append(class_acc['valid'])
                test_acc_list.append(class_acc['test'])

            except FileNotFoundError:
                # print('File not found: ', class_repeat_path)
                continue

        result_list.extend([np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list)])

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


In [ ]:
attack_type_list = ['stat', 'black']
for class_model in class_model_list:
    print('\''+class_model+'\',')

for class_model in class_model_list:
    attack_path = os.path.join(base_path, dataset, 'attacker', class_model)

    result_list = []
    for recon_type in ['full_z', 'content_z', 'style_z']:
        stat_acc_list = []
        black_acc_list = []
        # white_acc_list = []
        for repeat in range(5):
            for attack_type in attack_type_list:
                attack_repeat_path = os.path.join(attack_path, recon_type, 'repeat{}'.format(repeat))
                try:
                    attack_acc = np.load(os.path.join(attack_repeat_path, attack_type, 'acc.npy'), allow_pickle=True)
                    if attack_type == 'stat':
                        stat_acc_list.append(attack_acc)
                    elif attack_type == 'black':
                        black_acc_list.append(attack_acc.item()['test'])
                    # elif attack_type == 'white':
                    #     white_acc_list.append(attack_acc.item()['test'])

                except FileNotFoundError:
                    print('File not found: ', attack_repeat_path)
                    continue

        result_list.extend([np.average(stat_acc_list), np.average(black_acc_list)])

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


In [ ]:
classifier_with_raw_data = 'original_setsize2000_FCNClassifierA'
class_path = os.path.join(base_path, dataset, 'classifier', classifier_with_raw_data)
attack_path = os.path.join(base_path, dataset, 'attacker', classifier_with_raw_data)
train_acc_list = []
valid_acc_list = []
test_acc_list = []
stat_acc_list = []
black_acc_list = []
for repeat in range(5):
    try:
        class_repeat_path = os.path.join(class_path, 'repeat{}'.format(repeat))
        attack_repeat_path = os.path.join(attack_path, 'repeat{}'.format(repeat))

        class_acc = np.load(os.path.join(class_repeat_path, 'acc.npy'), allow_pickle=True).item()
        train_acc_list.append(class_acc['train'])
        valid_acc_list.append(class_acc['valid'])
        test_acc_list.append(class_acc['test'])

        for attack_type in attack_type_list:
            attack_acc = np.load(os.path.join(attack_repeat_path, attack_type, 'acc.npy'), allow_pickle=True)
            if attack_type == 'stat':
                stat_acc_list.append(attack_acc)
            elif attack_type == 'black':
                black_acc_list.append(attack_acc.item()['test'])

    except FileNotFoundError:
        print('File not found: ', class_repeat_path)
        continue

print('{:.4f} {:.4f} {:.4f} {:.4f} {:.4f}'.format(
    np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list),
    np.average(stat_acc_list), np.average(black_acc_list),
))
# print(np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list))
# print(np.average(stat_acc_list), np.average(black_acc_list), np.average(test_acc_list))

